In [1]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

import env

## Clustering - DBSCAN
### - Use DBSCAN to detect anomalies in other products from the customers dataset. The database name is grocery_db.
### - Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.
### - Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

## Acquire

In [2]:
# Acquiring the url from Codeup Data Science sql server and turning it into a dataframe
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/grocery_db'

query = '''
SELECT * FROM grocery_customers
     
'''
df = pd.read_sql(query, url)

In [3]:
# let's examine the head of the dataframe
df.head(2)

,customer_id,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,0,2,3,12669,9656,7561,214,2674,1338
1,1,2,3,7057,9810,9568,1762,3293,1776


In [4]:
# sanity check for the shape of the df:
df.shape

(440, 9)

In [5]:
#checking the dataframe for nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   customer_id       440 non-null    int64
 1   Channel           440 non-null    int64
 2   Region            440 non-null    int64
 3   Fresh             440 non-null    int64
 4   Milk              440 non-null    int64
 5   Grocery           440 non-null    int64
 6   Frozen            440 non-null    int64
 7   Detergents_Paper  440 non-null    int64
 8   Delicassen        440 non-null    int64
dtypes: int64(9)
memory usage: 31.1 KB


## Prepare

In [6]:
grocery = pd.read_sql(query, url)

In [7]:
# Cluster on all or a selected subset of features
grocery_milk_fresh = grocery[["Grocery", "Milk", "Fresh"]]
grocery_milk_fresh.head(2)


,Grocery,Milk,Fresh
0,7561,9656,12669
1,9568,9810,7057


In [8]:
# Make the scaler
scaler = MinMaxScaler()

# Fit the scaler
scaler.fit(grocery_milk_fresh)

# Use the scaler
grocery_milk_fresh = scaler.transform(grocery_milk_fresh)
grocery_milk_fresh


array([[0.08146416, 0.13072723, 0.11294004],
       [0.10309667, 0.13282409, 0.06289903],
       [0.08278992, 0.11918086, 0.05662161],
       ...,
       [0.32594285, 0.21013575, 0.1295431 ],
       [0.02402535, 0.02622442, 0.091727  ],
       [0.02702178, 0.02237109, 0.02482434]])

In [9]:
#after splitting now we can create a DBSCAN model by creating the object first
# Make the object
dbsc = DBSCAN(eps = .10, min_samples = 20)

# Fit the object
dbsc.fit(grocery_milk_fresh)


DBSCAN(eps=0.1, min_samples=20)

In [10]:
# Now, let's add the scaled value columns back onto the dataframe
columns = ["Grocery", "Milk", "Fresh"]
scaled_columns = ["Scaled_" + column for column in columns]

# Save a copy of the original dataframe
original_df = df.copy()

# Create a dataframe containing the scaled values
scaled_df = pd.DataFrame(grocery , columns=scaled_columns)

# Merge the scaled and non-scaled values into one dataframe
#df = df.merge(grocery , on=df.index)
#df = df.drop(columns=['key_0'])
df.head()


,customer_id,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,0,2,3,12669,9656,7561,214,2674,1338
1,1,2,3,7057,9810,9568,1762,3293,1776
2,2,2,3,6353,8808,7684,2405,3516,7844
3,3,1,3,13265,1196,4221,6404,507,1788
4,4,2,3,22615,5410,7198,3915,1777,5185


In [11]:
# Now, let's add the scaled value columns back onto the dataframe
columns = ["Grocery", "Milk", "Fresh"]
scaled_columns = ["Scaled_" + column for column in columns]

# Save a copy of the original dataframe
original_df = df.copy()

# Create a dataframe containing the scaled values
scaled_df = pd.DataFrame(grocery_milk_fresh, columns=scaled_columns)

# Merge the scaled and non-scaled values into one dataframe
#df = df.merge(scaled_df, on=df.index)
#df = df.drop(columns=['key_0'])
df.head()


,customer_id,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,0,2,3,12669,9656,7561,214,2674,1338
1,1,2,3,7057,9810,9568,1762,3293,1776
2,2,2,3,6353,8808,7684,2405,3516,7844
3,3,1,3,13265,1196,4221,6404,507,1788
4,4,2,3,22615,5410,7198,3915,1777,5185


In [12]:
# Get labels from cluster model

df['labels'] = dbsc.labels_
# Count
df.labels.value_counts()

 0    409
-1     31
Name: labels, dtype: int64

In [14]:
#sns.scatterplot(df.Grocery, df.Detergents_paper, hue = df.labels)
#plt.show()

#sns.scatterplot(df.Grocery, df.frozen, hue = df.labels)
#plt.show()